Índices invertidos e busca booleana
===================================

_Flávio Codeço Coelho (Com a contribuição dos alunos do curso de Sistemas de recuperação de Informações da EMAp)_

Nesta prática, vamos contruir um indice invertido e uma máquina de busca booleana simples.

Para agilizar nosso trabalho, vamos utilizar a biblioteca [NLTK](http://nltk.org) para processamento de linguagem natural.

In [1]:
import nltk

Em seguida vamos importar mais coisas necessárias para o nosso trabalho. Note que estamos baixando a obra completa de Machado de Assis, com a qual iremos alimentar nosso índice.

In [2]:
from nltk.corpus import machado
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
import string
from collections import defaultdict
from nltk.stem.snowball import PortugueseStemmer

Vamos também baixar o banco de *stopwords* do NLTK. Stop words são um conjunto de palavras que normalmente carregam baixo conteúdo semântico e portanto não são alvo de buscas.

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fccoelho/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Lendo o texto *puro* dos livros de Machado:

In [5]:
textos = [machado.raw(id) for id in machado.fileids()]
len(textos)

246

Carregando a  lista de stopwords em lingua portuguesa para limpeza dos textos. Note que é preciso trazer as palavras para *UTF-8* antes de usá-las.

In [9]:
swu = stopwords.words('portuguese') + list (string.punctuation)
#swu = [word.decode('utf8') for word in sw]

In [10]:
stopwords.words('portuguese')
list (string.punctuation)

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

Um outro ingrediente essencial é um stemmer para a nossa língua. O Stemmer reduz as palavras a uma abreviação que se aproxima da "raiz" da palavra.

In [11]:
stemmer = PortugueseStemmer()

Preparando o Texto
------------------

Na célula abaixo, vamos normalizar os nossos textos trazendo todas as palavras para caixa baixa e abreviando-as de forma a deixar apenas as suas raízes. Neste passo, removeremos também as *stopwords*. Tenha paciência, esta análise vai levar algum tempo...

In [12]:
textos_limpos = []
for texto in textos:
    tlimpo = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
    textos_limpos.append(tlimpo)

Vejamos uma amostra do resultado:

In [13]:
textos_limpos[0][150:160]

[u'uma',
 u'tal',
 u'miss',
 u'doll',
 u'dev',
 u'ter',
 u'poet',
 u'tennyson',
 u'cor',
 u'ler']

Construindo um Índice Invertido
-------------------------------

De posse da nossa lista de termos *normalizados*, podemos agora construir o nosso índice invertido.

In [14]:
indice = defaultdict(lambda:set([]))
for tid,t in enumerate(textos_limpos):
    for term in t:
        indice[term].add(tid)

Podemos verificar a estrutura interna do nosso índice, procurando por uma palavra:

In [16]:
indice[stemmer.stem(u"Salário")]

{3, 51, 77, 165, 182, 192, 193, 219, 220, 222, 232}

In [20]:
print stemmer.stem(u'Salarial')

salarial


Vamos ver o contexto em que a palavra *Salário* ocorre em um dos textos

In [25]:
nltk.Text(WordPunctTokenizer().tokenize(textos[192])).concordance(u"Salário")

Displaying 3 of 3 matches:
eio , porém , um dia em que acharam salário , e parece que bom salário . Quem o
 acharam salário , e parece que bom salário . Quem os contratou ? Quem é que fo
nhor ; eu só levo escritura .  Que salário pede por eles ?  Nenhum , pela min


In [96]:
def busca(consulta):
    """
    A construção de uma função de busca é deixada com exercício ao leitor
    """
    pass
    
    

Mas já podemos utilizar nosso índice diretamente com alguns termos e verificar como o mesmo é eficiente. 

In [29]:
%time
results = indice[stemmer.stem('nacional')]&indice[stemmer.stem('perdi')] - indice[stemmer.stem('campo')]
#results

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


Para um exame mais preciso do tempo de execução da nossa consulta, podemos usar a mágica do *%%timeit*

In [30]:
%%timeit
results = indice[stemmer.stem('nacional')]&indice[stemmer.stem('perdi')] - indice[stemmer.stem('campo')]
#results

10000 loops, best of 3: 79.5 µs per loop
